In [18]:
import pandas as pd
import sqlite3
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s:%(message)s")


In [19]:
def extract_excel(path):
    df = pd.read_excel(path, engine='openpyxl')
    logging.info(f"Extracted {len(df)} rows from {path}")
    return df


In [20]:
def transform(df):
    df = df.dropna(subset=['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'])
    df = df[df['Quantity'] > 0]
    df = df[df['UnitPrice'] > 0]
    df['TotalSales'] = df['Quantity'] * df['UnitPrice']
    logging.info(f"After transform: {len(df)} rows remain")
    return df


In [21]:
def build_dimensions(df):
    # Product
    prod_df = df[['StockCode', 'Description', 'UnitPrice']].drop_duplicates().reset_index(drop=True)
    prod_df['product_id'] = prod_df.index + 1

    # Customer
    cust_df = df.groupby(['CustomerID', 'Country']).agg(
        total_purchases=('TotalSales', 'sum'),
        join_date=('InvoiceDate', 'min')
    ).reset_index()
    cust_df['customer_id'] = cust_df.index + 1

    # Time
    time_df = pd.DataFrame({'date': pd.to_datetime(df['InvoiceDate']).dt.date.unique()})
    time_df['day'] = pd.to_datetime(time_df['date']).dt.day
    time_df['month'] = pd.to_datetime(time_df['date']).dt.month
    time_df['quarter'] = pd.to_datetime(time_df['date']).dt.quarter
    time_df['year'] = pd.to_datetime(time_df['date']).dt.year
    time_df['is_weekend'] = pd.to_datetime(time_df['date']).dt.weekday >= 5
    time_df['time_id'] = time_df.index + 1

    return prod_df, cust_df, time_df


In [22]:
def build_fact(df, prod_df, cust_df, time_df):
    fact_df = df.merge(prod_df, on=['Description', 'UnitPrice'], how='left')
    fact_df = fact_df.merge(cust_df, left_on=['CustomerID', 'Country'], right_on=['CustomerID', 'Country'], how='left')

    fact_df['date_only'] = pd.to_datetime(fact_df['InvoiceDate']).dt.date
    fact_df = fact_df.merge(time_df[['date', 'time_id']], left_on='date_only', right_on='date', how='left')

    fact_df = fact_df[['InvoiceNo', 'product_id', 'customer_id', 'time_id', 'Quantity', 'UnitPrice', 'TotalSales', 'Country']]
    return fact_df


In [23]:
def load_to_sqlite(prod_df, cust_df, time_df, fact_df, db_path="retail_dw.db", schema_path="dw_schema.sql"):
    conn = sqlite3.connect(db_path)

    # Drop tables if they exist
    conn.executescript("""
    DROP TABLE IF EXISTS SalesFact;
    DROP TABLE IF EXISTS Time;
    DROP TABLE IF EXISTS Product;
    DROP TABLE IF EXISTS Customer;
    """)

    # Load schema
    with open(schema_path, 'r') as f:
        schema_sql = f.read()
    conn.executescript(schema_sql)

    # Load data
    prod_df.rename(columns={'StockCode':'product_id', 'Description':'product_name', 'UnitPrice':'unit_price'}) \
           .to_sql('Product', conn, if_exists='append', index=False)

    cust_df.rename(columns={'CustomerID':'customer_name', 'Country':'country'}) \
           .to_sql('Customer', conn, if_exists='append', index=False)

    time_df[['time_id','date','day','month','quarter','year','is_weekend']] \
           .to_sql('Time', conn, if_exists='append', index=False)

    fact_df.rename(columns={'InvoiceNo':'invoice_no', 'Quantity':'quantity', 'UnitPrice':'unit_price', 'TotalSales':'total_sales', 'Country':'country'}) \
           .to_sql('SalesFact', conn, if_exists='append', index=False)

    conn.commit()

    # --- PREVIEW SAMPLE ROWS ---
    logging.info("Sample rows from each table:")
    for table in ["Customer", "Product", "Time", "SalesFact"]:
        sample_df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5;", conn)
        print(f"\n--- {table} ---")
        print(sample_df)

    conn.close()
    logging.info("Database recreated, loaded, and verified successfully.")


In [24]:
def run_etl(excel_path):
    df_raw = extract_excel(excel_path)
    df_trans = transform(df_raw)
    prod_df, cust_df, time_df = build_dimensions(df_trans)
    fact_df = build_fact(df_trans, prod_df, cust_df, time_df)
    load_to_sqlite(prod_df, cust_df, time_df, fact_df)

# Run and verify
run_etl(r"C:\Users\Administrator\OneDrive\Desktop\DSA2040_Practical_Exam_Rita940\OnlineRetail.xlsx")


2025-08-13 18:15:01,862 INFO:Extracted 541909 rows from C:\Users\Administrator\OneDrive\Desktop\DSA2040_Practical_Exam_Rita940\OnlineRetail.xlsx
2025-08-13 18:15:02,144 INFO:After transform: 397884 rows remain
2025-08-13 18:15:05,938 INFO:Sample rows from each table:
2025-08-13 18:15:05,954 INFO:Database recreated, loaded, and verified successfully.



--- Customer ---
   customer_id customer_name         country  total_purchases  \
0            1       12346.0  United Kingdom         77183.60   
1            2       12347.0         Iceland          4310.00   
2            3       12348.0         Finland          1797.24   
3            4       12349.0           Italy          1757.55   
4            5       12350.0          Norway           334.40   

             join_date  
0  2011-01-18 10:01:00  
1  2010-12-07 14:57:00  
2  2010-12-16 19:09:00  
3  2011-11-21 09:51:00  
4  2011-02-02 16:01:00  

--- Product ---
   product_id                         product_name category subcategory  \
0           1   WHITE HANGING HEART T-LIGHT HOLDER     None        None   
1           2                  WHITE METAL LANTERN     None        None   
2           3       CREAM CUPID HEARTS COAT HANGER     None        None   
3           4  KNITTED UNION FLAG HOT WATER BOTTLE     None        None   
4           5       RED WOOLLY HOTTIE WHITE HEART